# Import Libraries

In [12]:
!pip install mediapipe opencv-python ipywidgets tqdm numpy



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Body Type Detection

In [3]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def take_photo(filename='captured_image.jpg'):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    print("Press SPACE to capture the photo.")
    print("Press ESC to exit without capturing.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        cv2.imshow('Press SPACE to capture', frame)
        key = cv2.waitKey(1)
        
        if key % 256 == 27:  # ESC
            print("Escape hit, closing...")
            frame = None
            break
        elif key % 256 == 32:  # SPACE
            cv2.imwrite(filename, frame)
            print(f"Image captured and saved as {filename}")
            break

    cap.release()
    cv2.destroyAllWindows()
    return frame

def detect_body_type(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(image_rgb)

    if not results.pose_landmarks:
        return "Unknown", None

    landmarks = results.pose_landmarks.landmark
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    shoulder_width = abs(left_shoulder.x - right_shoulder.x)
    hip_width = abs(left_hip.x - right_hip.x)

    ratio = shoulder_width / hip_width

    # Draw landmarks on the image
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
    )

    if ratio > 1.2:
        return "Fit", annotated_image
    elif ratio > 0.9:
        return "Thin", annotated_image
    elif ratio > 0.7:
        return "Fat", annotated_image
    else:
        return "Obese", annotated_image

# Run the whole thing
frame = take_photo()

if frame is not None:
    body_type, annotated_frame = detect_body_type(frame)
    print(f"Detected Body Type: {body_type}")
    
    if annotated_frame is not None:
        cv2.imshow('Pose Landmarks', annotated_frame)
        cv2.waitKey(0)
        
        # Save the annotated image
        cv2.imwrite('annotated_image.jpg', annotated_frame)
        print("Annotated image saved as 'annotated_image.jpg'.")
        
        cv2.destroyAllWindows()
else:
    print("No image captured.")


Press SPACE to capture the photo.
Press ESC to exit without capturing.
Image captured and saved as captured_image.jpg


c:\Users\Iam_U\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Detected Body Type: Fit
Annotated image saved as 'annotated_image.jpg'.


In [9]:
# Exercise map based on body type
exercise_map = {
    'Thin': ['Pushups', 'Burpees', 'Squats'],
    'Fit': ['Pushups', 'Plank', 'Pullups'],
    'Fat': ['Seated Bicep Curls', 'Wall Pushups'],
    'Obese': ['Chair Squats', 'Wall Pushups']
}

# Assuming `body_type` is detected already from the previous part of your code
# Example usage:
# body_type = "Fit"  # For example, this could be the result from `detect_body_type`

# Get the suggested exercises based on body type
suggested_exercises = exercise_map.get(body_type, [])
print(f"Suggested Exercises for {body_type}: {suggested_exercises}")


Suggested Exercises for Fit: ['Pushups', 'Plank', 'Pullups']


# Exercise Selection

In [10]:
import tkinter as tk
from tkinter import ttk

# Exercise map based on body type
exercise_map = {
    'Thin': ['Pushups', 'Burpees', 'Squats'],
    'Fit': ['Pushups', 'Plank', 'Pullups'],
    'Fat': ['Seated Bicep Curls', 'Wall Pushups'],
    'Obese': ['Chair Squats', 'Wall Pushups']
}

# Assuming `body_type` is detected already from the previous part of your code
body_type = "Fit"  # Example: this could be the result from `detect_body_type`

# Get suggested exercises for the detected body type
suggested_exercises = exercise_map.get(body_type, [])

# Function to get the selected exercise
def get_selected_exercise():
    selected_exercise = exercise_choice.get()
    print(f"Selected Exercise: {selected_exercise}")

# Create the main window
root = tk.Tk()
root.title("Select Exercise")

# Create the dropdown menu
exercise_choice = ttk.Combobox(root, values=suggested_exercises)
exercise_choice.set(suggested_exercises[0])  # Set default value
exercise_choice.pack(pady=20)

# Create a button to get the selected exercise
select_button = tk.Button(root, text="Select Exercise", command=get_selected_exercise)
select_button.pack(pady=10)

# Run the Tkinter event loop
root.mainloop()


Selected Exercise: Pushups


# Debugging

In [8]:
import cv2
import numpy as np

# Function to capture webcam frame (live in loop)
def live_webcam_feed():
    cap = cv2.VideoCapture(0)  # Open the webcam
    if not cap.isOpened():
        print("Error: Could not access webcam.")
        return None

    while True:
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Failed to grab frame")
            break

        # Display the captured frame
        cv2.imshow("Webcam Feed", frame)

        # Debugging: Show the frame dimensions to track the webcam status
        print(f"Captured frame dimensions: {frame.shape}")

        # Exit the loop if the user presses 'q'
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to stop the webcam feed
            print("Exiting webcam feed...")
            break

    cap.release()  # Release the webcam
    cv2.destroyAllWindows()  # Close all OpenCV windows

# Start webcam feed and show captured frame
print("Starting live webcam feed...")
live_webcam_feed()


Starting live webcam feed...
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (480, 640, 3)
Captured frame dimensions: (

# Exercise Learning

Detection Feedback

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle for push-up stage detection
def calculate_angle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])
    
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Push-up stage detection
def pushup_stage(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

    if angle > 160:
        return "Up"
    elif angle < 90:
        return "Down"
    else:
        return "Transition"

# Function to capture webcam frame (live in loop)
def live_webcam_feed():
    cap = cv2.VideoCapture(0)  # Open the webcam
    if not cap.isOpened():
        print("Error: Could not access webcam.")
        return None

    expected_stage = "Down"  # Start with the "Down" stage
    failure_counter = 0
    max_failures = 3  # Number of failures allowed before suggesting an easier version
    completed = False
    step_start_time = time.time()  # Initialize timer for the first step
    step_passed_time = None  # Time to track when step passes

    while not completed:
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the frame to RGB (required by MediaPipe Pose)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # Process the frame if pose landmarks are detected
        if results.pose_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Detect push-up stage
            current_stage = pushup_stage(results.pose_landmarks.landmark)

            # Display the detected stage
            cv2.putText(frame, f"Suggestion: {current_stage}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

            # Show the step passed message for 3 seconds
            if current_stage == expected_stage and step_passed_time is None:
                step_passed_time = time.time()  # Start the step passed timer
                print(f"Step Passed! Expected: {expected_stage}")
            
            # If step is passed, display for 3 seconds and reset
            if step_passed_time is not None and time.time() - step_passed_time <= 3:
                cv2.putText(frame, "Step Passed!", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            elif step_passed_time is not None and time.time() - step_passed_time > 3:
                # Reset the step after 3 seconds
                step_passed_time = None
                failure_counter = 0  # Reset failure counter
                expected_stage = "Up" if expected_stage == "Down" else "Down"  # Switch stages
                step_start_time = time.time()  # Reset the timer

            # Check if current stage matches the expected stage
            if current_stage == expected_stage:
                print("Step passed!")
                failure_counter = 0
                expected_stage = "Up" if expected_stage == "Down" else "Down"  # Switch stages
                step_start_time = time.time()  # Reset the timer when step is passed
            else:
                elapsed_time = time.time() - step_start_time
                if elapsed_time >= 10:  # 10 seconds window expired
                    failure_counter += 1
                    print(f"Failure {failure_counter}")

                    # After 3 failures, suggest easier version
                    if failure_counter >= max_failures:
                        cv2.putText(frame, "Try an easier version!", (30, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                        completed = False  # End the session after too many failures
                else:
                    # Show the remaining time for the user to complete the step
                    remaining_time = 10 - int(elapsed_time)
                    # cv2.putText(frame, f"Time Remaining: {remaining_time}s", (30, 150),
                                # cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        else:
            # In case no person is detected
            cv2.putText(frame, "No Person Detected", (30, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        # Display the frame with annotations
        # Display timer in the upper-right corner
        elapsed_time = time.time() - step_start_time
        remaining_time = 10 - int(elapsed_time)
        cv2.putText(frame, f"{remaining_time}s", (frame.shape[1] - 100, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        # Display the frame with annotations
        cv2.imshow("Webcam Feed with Push-up Detection", frame)

        # Exit the loop if the user presses 'q' (Allow user to quit at any time)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to stop the webcam feed
            print("Exiting webcam feed...")
            break

    cap.release()  # Release the webcam
    cv2.destroyAllWindows()  # Close all OpenCV windows

# Start webcam feed and show captured frame with pose detection
print("Starting live webcam feed with push-up step-by-step detection...")
live_webcam_feed()


Starting live webcam feed with push-up step-by-step detection...
Step Passed! Expected: Down
Step passed!
Step passed!
Failure 1
Failure 2
Failure 3
Failure 4
Failure 5
Failure 6
Failure 7
Failure 8
Failure 9
Failure 10
Failure 11
Failure 12
Failure 13
Failure 14
Failure 15
Failure 16
Failure 17
Failure 18
Failure 19
Failure 20
Failure 21
Failure 22
Failure 23
Failure 24
Failure 25
Failure 26
Failure 27
Failure 28
Failure 29
Failure 30
Failure 31
Failure 32
Failure 33
Failure 34
Failure 35
Failure 36
Failure 37
Failure 38
Failure 39
Step Passed! Expected: Up
Step passed!
Step Passed! Expected: Up
Step passed!
Step Passed! Expected: Up
Step passed!
Step passed!
Step Passed! Expected: Down
Step passed!
Step passed!
Step passed!
Step Passed! Expected: Down
Step passed!
Step passed!
Exiting webcam feed...


Better Feedback

In [13]:
import cv2
import numpy as np
import mediapipe as mp
import time

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle for push-up stage detection
def calculate_angle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])
    
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Push-up stage detection
def pushup_stage(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

    if angle > 160:
        return "Go Up"  # When angle is above 160 degrees, the user is at the top (Go Up)
    elif angle < 90:
        return "Go Down"  # When angle is below 90 degrees, the user is at the bottom (Go Down)
    else:
        return "In Transition"  # Between "Go Down" and "Go Up"

# Function to capture webcam frame (live in loop)
def live_webcam_feed():
    cap = cv2.VideoCapture(0)  # Open the webcam
    if not cap.isOpened():
        print("Error: Could not access webcam.")
        return None

    expected_stage = "Go Down"  # Start with the "Go Down" stage
    failure_counter = 0
    max_failures = 3  # Number of failures allowed before suggesting an easier version
    completed = False
    step_start_time = time.time()  # Initialize timer for the first step
    step_passed_time = None  # Time to track when step passes

    while not completed:
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the frame to RGB (required by MediaPipe Pose)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # Process the frame if pose landmarks are detected
        if results.pose_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Detect push-up stage
            current_stage = pushup_stage(results.pose_landmarks.landmark)

            # Display the detected stage
            cv2.putText(frame, f"Detected: {current_stage}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

            # Show the step passed message for 3 seconds
            if current_stage == expected_stage and step_passed_time is None:
                step_passed_time = time.time()  # Start the step passed timer
                print(f"Step Passed! Expected: {expected_stage}")
            
            # If step is passed, display for 3 seconds and reset
            if step_passed_time is not None and time.time() - step_passed_time <= 3:
                cv2.putText(frame, "Step Passed!", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            elif step_passed_time is not None and time.time() - step_passed_time > 3:
                # Reset the step after 3 seconds
                step_passed_time = None
                failure_counter = 0  # Reset failure counter
                expected_stage = "Go Up" if expected_stage == "Go Down" else "Go Down"  # Switch stages
                step_start_time = time.time()  # Reset the timer

            # Check if current stage matches the expected stage
            if current_stage == expected_stage:
                print("Step passed!")
                failure_counter = 0
                expected_stage = "Go Up" if expected_stage == "Go Down" else "Go Down"  # Switch stages
                step_start_time = time.time()  # Reset the timer when step is passed
            else:
                elapsed_time = time.time() - step_start_time
                if elapsed_time >= 10:  # 10 seconds window expired
                    failure_counter += 1
                    print(f"Failure {failure_counter}")

                    # After 3 failures, suggest easier version
                    if failure_counter >= max_failures:
                        cv2.putText(frame, "Try an easier version!", (30, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                        completed = False  # End the session after too many failures
                else:
                    # Show the remaining time for the user to complete the step
                    remaining_time = 10 - int(elapsed_time)
                    # cv2.putText(frame, f"Time Remaining: {remaining_time}s", (30, 150),
                                # cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        else:
            # In case no person is detected
            cv2.putText(frame, "No Person Detected", (30, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        # Display the frame with annotations
        # Display timer in the upper-right corner
        elapsed_time = time.time() - step_start_time
        remaining_time = 10 - int(elapsed_time)
        cv2.putText(frame, f"{remaining_time}s", (frame.shape[1] - 100, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        # Display the frame with annotations
        cv2.imshow("Webcam Feed with Push-up Detection", frame)

        # Exit the loop if the user presses 'q' (Allow user to quit at any time)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to stop the webcam feed
            print("Exiting webcam feed...")
            break

    cap.release()  # Release the webcam
    cv2.destroyAllWindows()  # Close all OpenCV windows

# Start webcam feed and show captured frame with pose detection
print("Starting live webcam feed with push-up step-by-step detection...")
live_webcam_feed()


Starting live webcam feed with push-up step-by-step detection...
Step Passed! Expected: Go Down
Step passed!
Step passed!
Step passed!
Step passed!
Step passed!
Step Passed! Expected: Go Down
Step passed!
Step passed!
Step passed!
Step passed!
Step Passed! Expected: Go Up
Step passed!
Step passed!
Step passed!
Step Passed! Expected: Go Up
Step passed!
Step passed!
Failure 1
Failure 2
Failure 3
Failure 4
Failure 5
Failure 6
Failure 7
Failure 8
Failure 9
Failure 10
Failure 11
Failure 12
Failure 13
Failure 14
Failure 15
Failure 16
Failure 17
Failure 18
Failure 19
Failure 20
Failure 21
Failure 22
Failure 23
Failure 24
Failure 25
Failure 26
Failure 27
Failure 28
Failure 29
Failure 30
Failure 31
Failure 32
Failure 33
Failure 34
Failure 35
Failure 36
Failure 37
Failure 38
Failure 39
Failure 40
Failure 41
Failure 42
Failure 43
Failure 44
Failure 45
Failure 46
Failure 47
Failure 48
Failure 49
Failure 50
Failure 51
Failure 52
Failure 53
Failure 54
Failure 55
Failure 56
Failure 57
Failure 58
Fail

Opens Up menu of Easier Exercises

In [3]:
import cv2
import numpy as np
import mediapipe as mp
import time
import ipywidgets as widgets
from IPython.display import display

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)
mp_drawing = mp.solutions.drawing_utils

# Define easier exercises
easier_exercises = {
    "Chair Squats": "Chair Squats are a great low-impact exercise for beginners.",
    "Wall Pushups": "Wall pushups are a great way to build upper body strength gradually.",
    "Seated Bicep Curls": "Seated bicep curls help to strengthen the arms in a seated position.",
}

# Function to calculate angle for push-up stage detection
def calculate_angle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])
    
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Push-up stage detection
def pushup_stage(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

    if angle > 160:
        return "Go Up"  # When angle is above 160 degrees, the user is at the top (Go Up)
    elif angle < 90:
        return "Go Down"  # When angle is below 90 degrees, the user is at the bottom (Go Down)
    else:
        return "In Transition"  # Between "Go Down" and "Go Up"

# Function to capture webcam frame (live in loop)
def live_webcam_feed():
    cap = cv2.VideoCapture(0)  # Open the webcam
    if not cap.isOpened():
        print("Error: Could not access webcam.")
        return None

    expected_stage = "Go Down"  # Start with the "Go Down" stage
    failure_counter = 0
    max_failures = 3  # Number of failures allowed before suggesting an easier version
    completed = False
    step_start_time = time.time()  # Initialize timer for the first step
    step_passed_time = None  # Time to track when step passes

    while not completed:
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the frame to RGB (required by MediaPipe Pose)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # Process the frame if pose landmarks are detected
        if results.pose_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Detect push-up stage
            current_stage = pushup_stage(results.pose_landmarks.landmark)

            # Display the detected stage
            cv2.putText(frame, f"Detected: {current_stage}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

            # Show the step passed message for 3 seconds
            if current_stage == expected_stage and step_passed_time is None:
                step_passed_time = time.time()  # Start the step passed timer
                print(f"Step Passed! Expected: {expected_stage}")
            
            # If step is passed, display for 3 seconds and reset
            if step_passed_time is not None and time.time() - step_passed_time <= 3:
                cv2.putText(frame, "Step Passed!", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            elif step_passed_time is not None and time.time() - step_passed_time > 3:
                # Reset the step after 3 seconds
                step_passed_time = None
                failure_counter = 0  # Reset failure counter
                expected_stage = "Go Up" if expected_stage == "Go Down" else "Go Down"  # Switch stages
                step_start_time = time.time()  # Reset the timer

            # Check if current stage matches the expected stage
            if current_stage == expected_stage:
                print("Step passed!")
                failure_counter = 0
                expected_stage = "Go Up" if expected_stage == "Go Down" else "Go Down"  # Switch stages
                step_start_time = time.time()  # Reset the timer when step is passed
            else:
                elapsed_time = time.time() - step_start_time
                if elapsed_time >= 10:  # 10 seconds window expired
                    failure_counter += 1
                    print(f"Failure {failure_counter}")

                    # After 3 failures, suggest easier version
                    if failure_counter >= max_failures:
                        cv2.putText(frame, "Try an easier version!", (30, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

                        # Display easier exercise selection menu
                        display_easier_exercises()
                        completed = True  # End the session after too many failures
                else:
                    # Show the remaining time for the user to complete the step
                    remaining_time = 10 - int(elapsed_time)
                    cv2.putText(frame, f"Time Remaining: {remaining_time}s", (30, 150),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        else:
            # In case no person is detected
            cv2.putText(frame, "No Person Detected", (30, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        # Display the frame with annotations
        # Display timer in the upper-right corner
        elapsed_time = time.time() - step_start_time
        remaining_time = 10 - int(elapsed_time)
        cv2.putText(frame, f"{remaining_time}s", (frame.shape[1] - 100, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        # Display the frame with annotations
        cv2.imshow("Webcam Feed with Push-up Detection", frame)

        # Exit the loop if the user presses 'q' (Allow user to quit at any time)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to stop the webcam feed
            print("Exiting webcam feed...")
            break

    cap.release()  # Release the webcam
    cv2.destroyAllWindows()  # Close all OpenCV windows

# Function to display easier exercises selection menu
def display_easier_exercises():
    print("Suggested Easier Exercises:")
    
    # Create dropdown menu for easier exercises
    exercise_choice = widgets.Dropdown(
        options=list(easier_exercises.keys()),
        description="Choose an Exercise:"
    )
    display(exercise_choice)

    # Button to confirm exercise choice
    button = widgets.Button(description="Start Selected Exercise")
    display(button)

    # Action when user clicks the button
    def on_button_clicked(b):
        selected_exercise = exercise_choice.value
        print(f"You selected: {selected_exercise}")
        print(f"Exercise Info: {easier_exercises[selected_exercise]}")
        start_exercise(selected_exercise)

    button.on_click(on_button_clicked)

# Function to simulate starting an easier exercise (For demonstration purposes)
def start_exercise(exercise):
    print(f"Starting the exercise: {exercise}")
    # You can add code here to load a new video, guide, or action for the exercise.
    # For now, this is just a placeholder.

# Start webcam feed and show captured frame with pose detection
print("Starting live webcam feed with push-up step-by-step detection...")
live_webcam_feed()


Starting live webcam feed with push-up step-by-step detection...


c:\Users\Iam_U\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Step Passed! Expected: Go Down
Step passed!
Step Passed! Expected: Go Down
Step passed!
Step passed!
Step Passed! Expected: Go Up
Step passed!
Step passed!
Step passed!
Step passed!
Step Passed! Expected: Go Down
Step passed!
Step passed!
Step passed!
Failure 1
Failure 2
Failure 3
Suggested Easier Exercises:


Dropdown(description='Choose an Exercise:', options=('Chair Squats', 'Wall Pushups', 'Seated Bicep Curls'), va…

Button(description='Start Selected Exercise', style=ButtonStyle())

PopUp Menu

# Earlier Logic

In [15]:
import cv2
import numpy as np
import time
import mediapipe as mp

# Setup Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)
mp_drawing = mp.solutions.drawing_utils

# Global variable to keep track of expected push-up stage
expected_stage = "Down"  # Start with the "Down" stage

# Function to capture webcam frame (live in loop)
def live_webcam_feed():
    global expected_stage  # Access the global expected_stage variable
    cap = cv2.VideoCapture(0)  # Open the webcam
    if not cap.isOpened():
        print("Error: Could not access webcam.")
        return None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Process the frame
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            current_stage = pushup_stage(results.pose_landmarks.landmark)
            # Draw landmarks on the frame
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Display detected and expected stage
            cv2.putText(frame, f"Detected: {current_stage}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            cv2.putText(frame, f"Expected: {expected_stage}", (30, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

            # Logic for counting/failure detection
            if current_stage == expected_stage:
                print("Step passed!")
                failure_counter = 0
                expected_stage = "Up" if expected_stage == "Down" else "Down"
            else:
                failure_counter += 1
                print(f"Failure {failure_counter}")

            if failure_counter >= max_failures:
                print("Suggesting easier version: Half Pushup!")
                break
        else:
            current_stage = "No Person"

        # Display the frame with annotations
        cv2.imshow("Live Push-Up Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to calculate angle for push-up stage detection
def calculate_angle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])
    
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Push-up stage detection
def pushup_stage(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

    if angle > 160:
        return "Up"
    elif angle < 90:
        return "Down"
    else:
        return "Transition"

# Main loop
failure_counter = 0
max_failures = 3

print("Starting live webcam feed...")
live_webcam_feed()


Starting live webcam feed...
Step passed!
Failure 1
Failure 2
Failure 3
Suggesting easier version: Half Pushup!


# Rep Counter

Didn't work

In [2]:
import cv2
from tqdm import tqdm

# Simulate or implement your pushup stage detector
def pushup_stage(frame):
    # Placeholder logic: You should replace this with your actual detection
    # For example, alternate between "Up" and "Down" based on frame count or posture detection
    import random
    return random.choice(["Up", "Down"])

# Capture image from webcam
def capture_image():
    ret, frame = cap.read()
    if not ret:
        raise Exception("Failed to capture image from webcam")
    return frame

# Set up camera
cap = cv2.VideoCapture(0)  # 0 is usually the default webcam

total_reps = 10
rep_count = 0
bar = tqdm(total=total_reps)

last_stage = None

try:
    while rep_count < total_reps:
        frame = capture_image()
        current_stage = pushup_stage(frame)

        if current_stage == "Down" and last_stage == "Up":
            rep_count += 1
            bar.update(1)
            print(f"Reps: {rep_count}")

        last_stage = current_stage

finally:
    cap.release()
    cv2.destroyAllWindows()
    bar.close()
    print("Workout complete!")


 20%|██        | 2/10 [00:00<00:02,  2.68it/s]

Reps: 1
Reps: 2


 40%|████      | 4/10 [00:00<00:01,  5.59it/s]

Reps: 3
Reps: 4


 60%|██████    | 6/10 [00:01<00:00,  5.55it/s]

Reps: 5
Reps: 6


 80%|████████  | 8/10 [00:01<00:00,  5.93it/s]

Reps: 7
Reps: 8
Reps: 9
Reps: 10


100%|██████████| 10/10 [00:02<00:00,  4.95it/s]

Workout complete!
